In [ ]:
!pip install -q "arize-phoenix==0.0.14rc0" 'arize[AutoEmbeddings, LLM_Evaluation]' 

In [ ]:
!pip install -q 'arize[AutoEmbeddings, LLM_Evaluation]' 

In [ ]:
!pip install wrapt

In [ ]:
from datetime import datetime, timedelta
import uuid
import pandas as pd

from arize.pandas.embeddings import EmbeddingGenerator, UseCases
from arize.pandas.generative.llm_evaluation import sacre_bleu, rouge
import phoenix as px

In [ ]:
df = pd.read_json("https://storage.googleapis.com/arize-assets/fixtures/Embeddings/arize-demo-models-data/GENERATIVE/summarization/cnn_news_summarization.json")
df.head()

In [ ]:
df['sacreBLEU_score'] = sacre_bleu(response_col=df["summary"], references_col=df["reference_summary"])
rouge_scores = rouge(response_col=df["summary"], references_col=df["reference_summary"], rouge_types=["rouge1", "rouge2", "rougeL", "rougeLsum"])
for rouge_type, scores in rouge_scores.items():
    df[f"{rouge_type}_score"] = scores

In [ ]:
df.head()

In [ ]:
generator = EmbeddingGenerator.from_use_case(
    use_case=UseCases.NLP.SUMMARIZATION,
    model_name="distilbert-base-uncased",
    tokenizer_max_length=512,
    batch_size=100
)

In [ ]:
df["document_vector"] = generator.generate_embeddings(text_col=df["document"])
df["summary_vector"] = generator.generate_embeddings(text_col=df["summary"])

In [ ]:
now_dt = datetime.now()
start_dt = now_dt - timedelta(days=15)
df["prediction_ts"] = pd.date_range(
    start=start_dt,
    end=now_dt,
    periods=len(df),
)

In [ ]:
df = pd.read_parquet('/Users/xandersong/Downloads/data.parquet')

In [ ]:
schema = px.Schema(
    timestamp_column_name="prediction_ts",
    tag_column_names=["sacreBLEU_score", "rouge1_score", "rouge2_score", "rougeL_score","rougeLsum_score"],
    prompt_column_names=px.EmbeddingColumnNames(
        vector_column_name="document_vector",
        raw_data_column_name="document"
    ),
    response_column_names=px.EmbeddingColumnNames(
        vector_column_name="summary_vector",
        raw_data_column_name="summary"
    )
)

In [ ]:
df.summary_vector.dtype.kind

In [ ]:
ds = px.Dataset(df, schema)

In [ ]:
px.launch_app(ds)

In [ ]:
df.to_parquet('/content/data.parquet', version='2.6')

In [ ]:
df.prediction_ts